In [2]:
# Initial imports
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
import seaborn as sns
%matplotlib inline
#import matplotlib as plt
whalepath=Path('whale_returns.csv')
sp500path=Path('sp500_history.csv')
algopath=Path('algo_returns.csv')




#def grab(symbol,command,period,start,end):
########################
myportfoliostonks=['TSLA','FCEL','LUV','SLB','BP']
for company in myportfoliostonks:
    exec('{} = pd.DataFrame()'.format(company))
    ticker=company
    tickerData=yf.Ticker(ticker)
    tickerDf = tickerData.history(period='1d',start='2020-12-20',end='2021-1-7').drop(columns=['Open','High','Low','Volume','Dividends','Stock Splits'])
    #tickerDf.drop(columns=['Open','High','Low','Volume','Dividends','Stock Splits'])
    print(tickerDf.shape)

#tickerData.calendar
#tickerData.recommendations
tickerData.info
######################

(11, 1)
(11, 1)
(11, 1)
(11, 1)
(11, 1)


{'zip': 'SW1Y 4PD',
 'sector': 'Energy',
 'fullTimeEmployees': 67600,
 'longBusinessSummary': 'BP PLC engages in the energy business worldwide. It operates through three segments: Upstream, Downstream, and Rosneft. The Upstream segment is involved in the oil and natural gas exploration, and field development and production; midstream transportation, storage, and processing; and marketing and trading of liquefied natural gas (LNG), biogas, power, and natural gas liquids (NGLs). This segment also engages in the ownership and management of crude oil and natural gas pipelines; processing facilities and export terminals; and LNG processing facilities and transportation, as well as NGLs processing business. The Downstream segment refines, manufactures, markets, transports, supplies, and trades in crude oil, petroleum, and petrochemical products and related services to wholesale and retail customers. It offers gasoline, diesel, and aviation fuels; lubricants, and related products and services

In [ ]:
myportfoliostonks=['TSLA','FCEL','LUV','SLB','BP']
for company in myportfoliostonks:
    ticker=company
    tickerData=yf.Ticker(ticker)
    tickerDf=tickerData.history(period='1d',start='2020-12-20',end='2021-1-7')
    tickerDf.drop(columns=['Open','High','Low','Volume','Dividends','Stock Splits'])
    print(tickerDF.head())


In [ ]:
# Reading whale returns
whale_df = pd.read_csv(whalepath
                        , index_col= 'Date'
                        , infer_datetime_format= True
                        , parse_dates= True
                    ).sort_index()


In [ ]:
# Count nulls
whale_df.isnull().sum()
# Drop nulls
whale_cleandf=whale_df.dropna()
# Check Data
whale_df.info()

In [ ]:
# Reading algorithmic returns
algo_df= pd.read_csv(algopath
                    , index_col='Date'
                    , infer_datetime_format=True
                    , parse_dates=True
                 ).sort_index()
                

In [ ]:
# Count nulls
algo_df.isnull().sum()
# Drop nulls
algo_cleandf= algo_df.dropna()
# Check Data/
algo_cleandf.info()

In [ ]:
# Reading S&P 500 Closing Prices
sp500df= pd.read_csv(sp500path
                    , index_col= 'Date'
                    , infer_datetime_format= True
                    , parse_dates= True
                    ).sort_index()

In [ ]:
# Count Nulls
sp500df.isnull().sum().sum()
# Fix Data Types
sp500df["Close"] = sp500df["Close"].str.replace("$", "").astype("float")

In [ ]:
# Calculate Daily Returns
sp500dfrtns=sp500df.pct_change()

In [ ]:
# Drop nulls & Rename `Close` Column to be specific to this portfolio.
sp500cleaned=sp500dfrtns.dropna().rename(columns={"Close": "S&P500 Returns"})
sp500cleaned.info()

In [ ]:
# Join Whale Returns, Algorithmic Returns, and the S&P 500 Returns into a single DataFrame with columns for each portfolio's returns.
combinedrtns = pd.concat([sp500cleaned,algo_cleandf,whale_cleandf], axis='columns', join='inner')
combinedrtns.info()

In [ ]:
# Plot daily returns of all portfolios
combinedrtns.plot(
            kind='line'
            , figsize=(20,20)
            , grid=True
            , subplots=True
            , fontsize=20
            , ylim=[-.06,.06]
            , rot=360
            , lw=2
        )
    

In [ ]:
# Calculate cumulative returns of all portfolios
cum_rets = (1 + combinedrtns).cumprod()
# Plot cumulative returns
cum_rets.plot(
        kind='line'
        , figsize=(25,10)
        , grid=True
        , title="Portfolio Returns"
        , ylabel="Total Rtns"
        , xlabel='Date\'s'
        , rot=360
        , fontsize=15
        , color=['red','g','y','b','orange','pink','black']
    )

In [ ]:
# Box plot to visually show risk
cum_rets.plot(
    kind='box'
    , figsize=(25,10)
    , grid=True 
    , title="Portfolio Returns"
    #, ylim=[-.2,.2]
    , ylabel="Total Rtns"
    , rot=360
    #""" fix this immmediatly """, whis = [0,0]
    #, fontsize=15
) 

In [ ]:
# Calculate the daily standard deviations of all portfolios
daily_std=combinedrtns.std().sort_values()
daily_std

In [ ]:
# my horrible attempt at making a more readable table to assess the most risky portfolios.
daily_stdchart= pd.concat([daily_std],axis='columns',join='outer')
bins = [0, 0.008553, 0.008555, 1]
group_names = ["Lower", "Benchmark", "Higher"]
daily_stdchart["Risk Level vs S&P500"] = pd.cut(daily_stdchart[0], bins, labels=group_names)
daily_stdchart.columns = ['STD','Risk Level']
daily_stdchart


In [ ]:
# Calculate  the daily standard deviation of S&P 500
daily_std['S&P500 Returns'].round(6)
# Determine which portfolios are riskier than the S&P 500
print(f'The following Portfolios are riskier than the SP500: \n\n '
      f'{daily_stdchart.iloc[5::]}')

In [ ]:
# Calculate the annualized standard deviation (252 trading days)
annual_std=daily_std*np.sqrt(252)
annual_std.sort_values(inplace=True)
annual_std.dtypes
annual_std

In [ ]:
# Calculate the rolling standard deviation for all portfolios using a 21-day window
rolltwentyone=combinedrtns.rolling(window=21).std()
# Plot the rolling standard deviation
rolltwentyone.plot(
              kind='line'
            , figsize=(40,20)
            , grid=True
            #, subplots=True
            , fontsize=12
            , ylim=[0,.035]
            , rot=360
            , lw=2
            , color=['r','b','b','g','g','g','g']
        )

In [ ]:
# Calculate the correlation
correlation=combinedrtns.corr()
# Display the correlation matrix
print(correlation.sum(axis=0).sort_values())
sns.set_theme(style='white')
sns.heatmap(correlation, vmin=0, vmax=1,annot=True,)


In [ ]:
### Calculate and Plot Beta for a chosen portfolio and the S&P 500
combinedrtns.keys()


In [ ]:
#.  how the hell do i make this condensed into a for or if statement...can i create a custom bdate range datetimeIndex and assign it to a index column in a empty data series, then concat the inner data onto the needed dates?



#1. Calculate and plot the rolling standard deviation for all portfolios using a 21-day window.
#2. Calculate and plot the correlation between each stock to determine which portfolios may mimick the S&P 500.
#3. Choose one portfolio, then calculate and plot the 60-day rolling beta between it and the S&P 500.
# Calculate covariance of a single portfolio--i didnt know which one i wanted to do 
algo_1_cov=combinedrtns['Algo 1'].cov(combinedrtns['S&P500 Returns'])
algo_2_cov=combinedrtns['Algo 2'].cov(combinedrtns['S&P500 Returns'])
soros_cov=combinedrtns['SOROS FUND MANAGEMENT LLC'].cov(combinedrtns['S&P500 Returns'])
paul_cov=combinedrtns['PAULSON & CO.INC. '].cov(combinedrtns['S&P500 Returns'])
tiger_cov=combinedrtns['TIGER GLOBAL MANAGEMENT LLC'].cov(combinedrtns['S&P500 Returns'])
buffet_cov=combinedrtns['BERKSHIRE HATHAWAY INC'].cov(combinedrtns['S&P500 Returns'])
# Calculate variance of S&P 500
variance=combinedrtns['S&P500 Returns'].var()
# Computing beta
algo_1_beta=algo_1_cov/variance
algo_2_beta=algo_2_cov/variance
soros_beta=soros_cov/variance
paul_beta=paul_cov/variance
tiger_beta=tiger_cov/variance
buffet_beta=buffet_cov/variance
# Calculate rolling covariance
roll_algo_1_cov = combinedrtns['Algo 1'].rolling(window=21).cov(combinedrtns['S&P500 Returns'])
roll_algo_2_cov = combinedrtns['Algo 2'].rolling(window=21).cov(combinedrtns['S&P500 Returns'])
roll_soros_cov = combinedrtns['SOROS FUND MANAGEMENT LLC'].rolling(window=21).cov(combinedrtns['S&P500 Returns'])
roll_paul_cov = combinedrtns['PAULSON & CO.INC. '].rolling(window=21).cov(combinedrtns['S&P500 Returns'])
roll_tiger_cov = combinedrtns['TIGER GLOBAL MANAGEMENT LLC'].rolling(window=21).cov(combinedrtns['S&P500 Returns'])
roll_buffet_cov = combinedrtns['BERKSHIRE HATHAWAY INC'].rolling(window=21).cov(combinedrtns['S&P500 Returns'])
# Calculate rolling variance of sp500
roll_variance = combinedrtns['S&P500 Returns'].rolling(window=60).var()
# calulating rolling betas
roll_algo_1_beta = roll_algo_1_cov / roll_variance
roll_algo_2_beta = roll_algo_2_cov / roll_variance
roll_soros_beta = roll_soros_cov / roll_variance
roll_paul_beta = roll_paul_cov / roll_variance
roll_tiger_beta = roll_tiger_cov / roll_variance
roll_buffet_beta = roll_buffet_cov / roll_variance
df=pd.concat([roll_algo_1_beta,roll_algo_2_beta,roll_soros_beta,roll_paul_beta,roll_tiger_beta,roll_buffet_beta], axis='columns', join='inner')
#ploting these 
ax = roll_algo_1_beta.plot(figsize=(20, 10)
                            , title='Rolling 60-Day Beta of Portfolios'
                            , grid = True
                            , rot=360);
roll_algo_2_beta.plot(ax=ax
                     , rot=360
                     , grid = True)
roll_soros_beta.plot(ax=ax
                     , rot=360
                     , grid = True)
roll_paul_beta.plot(ax=ax
                     , rot=360
                     , grid = True)
roll_tiger_beta.plot(ax=ax
                     , rot=360
                     , grid = True)
roll_buffet_beta.plot(ax=ax
                     , rot=360
                     , grid = True)
ax.legend(['Algo 1', 'Algo 2', 'SOROS FUND MANAGEMENT LLC',
              'PAULSON & CO.INC. ', 'TIGER GLOBAL MANAGEMENT LLC',
              'BERKSHIRE HATHAWAY INC'])
df.plot(kind='line').legend(['Algo 1', 'Algo 2', 'SOROS FUND MANAGEMENT LLC',
       'PAULSON & CO.INC. ', 'TIGER GLOBAL MANAGEMENT LLC',
       'BERKSHIRE HATHAWAY INC'])

## Rolling Statistics Challenge: Exponentially Weighted Average 

An alternative way to calculate a rolling window is to take the exponentially weighted moving average. This is like a moving window average, but it assigns greater importance to more recent observations. Try calculating the [`ewm`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html) with a 21-day half-life.

In [ ]:
# Use `ewm` to calculate the rolling window
#times=['2015-03-03':'2019-04-23':'1D']
df.index
dfewa=df.ewm(halflife='21 days',times=(df.index)).mean()
dfewa.columns=['Algo 1', 'Algo 2', 'SOROS FUND MANAGEMENT LLC',
       'PAULSON & CO.INC. ', 'TIGER GLOBAL MANAGEMENT LLC',
       'BERKSHIRE HATHAWAY INC']
dfewa.plot(subplots=True,figsize=(30,40),grid=True)


---

# Sharpe Ratios
In reality, investment managers and thier institutional investors look at the ratio of return-to-risk, and not just returns alone. After all, if you could invest in one of two portfolios, and each offered the same 10% return, yet one offered lower risk, you'd take that one, right?

### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [ ]:
# Annualized Sharpe Ratios
sharpie_ratio=((combinedrtns.mean())*252)/(combinedrtns.std()*np.sqrt(252))
sharpie_ratio.sort_values(inplace=True)
sharpie_ratio

In [ ]:
# Visualize the sharpe ratios as a bar plot
sharpie_ratio.plot(
                kind='bar'
                , rot=360
                , figsize=(30,10)
)

### Determine whether the algorithmic strategies outperform both the market (S&P 500) and the whales portfolios.

Write your answer here!

---

# Create Custom Portfolio

In this section, you will build your own portfolio of stocks, calculate the returns, and compare the results to the Whale Portfolios and the S&P 500. 

1. Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.
2. Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock
3. Join your portfolio returns to the DataFrame that contains all of the portfolio returns
4. Re-run the performance and risk analysis with your portfolio to see how it compares to the others
5. Include correlation analysis to determine which stocks (if any) are correlated

## Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.

For this demo solution, we fetch data from three companies listes in the S&P 500 index.

* `GOOG` - [Google, LLC](https://en.wikipedia.org/wiki/Google)

* `AAPL` - [Apple Inc.](https://en.wikipedia.org/wiki/Apple_Inc.)

* `COST` - [Costco Wholesale Corporation](https://en.wikipedia.org/wiki/Costco)

In [ ]:
# Reading data from 1st stock


In [ ]:
# Reading data from 2nd stock


In [ ]:
# Reading data from 3rd stock


In [ ]:
# Combine all stocks in a single DataFrame


In [ ]:
# Reset Date index


In [ ]:
# Reorganize portfolio data by having a column per symbol


In [ ]:
# Calculate daily returns

# Drop NAs

# Display sample data


## Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock

In [ ]:
# Set weights
weights = [1/3, 1/3, 1/3]

# Calculate portfolio return

# Display sample data


## Join your portfolio returns to the DataFrame that contains all of the portfolio returns

In [ ]:
# Join your returns DataFrame to the original returns DataFrame


In [ ]:
# Only compare dates where return data exists for all the stocks (drop NaNs)


## Re-run the risk analysis with your portfolio to see how it compares to the others

### Calculate the Annualized Standard Deviation

In [ ]:
# Calculate the annualized `std`


### Calculate and plot rolling `std` with 21-day window

In [ ]:
# Calculate rolling standard deviation

# Plot rolling standard deviation


### Calculate and plot the correlation

In [ ]:
# Calculate and plot the correlation


### Calculate and Plot Rolling 60-day Beta for Your Portfolio compared to the S&P 500

In [ ]:
# Calculate and plot Beta


### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [ ]:
# Calculate Annualzied Sharpe Ratios


In [ ]:
# Visualize the sharpe ratios as a bar plot


### How does your portfolio do?

Write your answer here!